In [ ]:

import torch 
from torch import nn 
import yfinance as yf
import talib as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
import pandas as pd 
from torch.utils.data import DataLoader
import numpy as np


In [2]:
#Finance Indicators 
f_indicators = ["MACD", "RSI", "CMO"," MOM", "Bollinger Bands", "SMA"]
ticker= "BTC-USD"
start_date = "2023-01-01"
end_date = "2025-12-31"

data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
close_price = data["Close"].values.flatten()
data.head(10)
close_price


[*********************100%***********************]  1 of 1 completed


array([16625.08007812, 16688.47070312, 16679.85742188, ...,
       86419.78125   , 87843.984375  , 86772.75      ], shape=(1082,))

In [3]:
data.isna().sum()

Price   Ticker 
Close   BTC-USD    0
High    BTC-USD    0
Low     BTC-USD    0
Open    BTC-USD    0
Volume  BTC-USD    0
dtype: int64

In [4]:
#Finance Indicators 
f_indicators = ["MACD", "RSI", "CMO"," MOM", "Bollinger Bands", "SMA"]
ticker= "BTC-USD"
start_date = "2023-01-01"
end_date = "2025-12-31"

data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
close_price = data["Close"].values.flatten()
data.head(10)
close_price

# MACD (TA-Lib returns 3 arrays)
macd, signal, hist = ta.MACD(close_price, fastperiod=12, slowperiod=26, signalperiod=9)

# Add to DataFrame
data["MACD"] = macd
data["Signal"] = signal
data["Hist"] = hist

# RSI
data["RSI"] = ta.RSI(close_price, timeperiod=14)

# CMO
data["CMO"] = ta.CMO(close_price, timeperiod=14)

# MOM
data["MOM"] = ta.MOM(close_price, timeperiod=14)

# Bollinger Bands
upper, middle, lower = ta.BBANDS(close_price, timeperiod=14, nbdevup=2, nbdevdn=2, matype=0)

# Add to DataFrame
data["BBU"] = upper    # Upper band
data["BBM"] = middle   # Middle band (SMA)
data["BBL"] = lower    # Lower band

# SMA
data["SMA"] = ta.SMA(close_price, timeperiod=14)
data.head(30)
data.isna().sum()

[*********************100%***********************]  1 of 1 completed


Price   Ticker 
Close   BTC-USD     0
High    BTC-USD     0
Low     BTC-USD     0
Open    BTC-USD     0
Volume  BTC-USD     0
MACD               33
Signal             33
Hist               33
RSI                14
CMO                14
MOM                14
BBU                13
BBM                13
BBL                13
SMA                13
dtype: int64

In [34]:
# Remove all rows with NaNs caused by indicator calculations
plot_data = data.dropna().copy()
plot_data = plot_data.copy()
plot_data.columns = [
    f"{lvl0}" if lvl1 else lvl0
    for lvl0, lvl1 in plot_data.columns
]

plot_data.index = pd.to_datetime(plot_data.index)  # ensure datetime index



In [35]:
plot_data

,Close,High,Low,Open,Volume,MACD,Signal,Hist,RSI,CMO,MOM,BBU,BBM,BBL,SMA
Date,,,,,,,,,,,,,,,
2023-02-03,23449.322266,23678.103516,23279.955078,23469.412109,27083066007,1658.371173,2013.486323,-355.115150,71.870229,43.740458,772.769531,23813.408952,23123.453404,22433.497856,23123.453404
2023-02-04,23331.847656,23556.949219,23291.794922,23446.320312,15639298538,1560.052550,1922.799569,-362.747019,70.054386,40.108773,554.222656,23832.374304,23163.040737,22493.707169,23163.040737
2023-02-05,22955.666016,23423.435547,22841.759766,23332.248047,19564262605,1435.235120,1825.286679,-390.051559,64.439754,28.879508,235.250000,23814.815753,23179.844308,22544.872863,23179.844308
2023-02-06,22760.109375,23119.279297,22692.025391,22954.021484,23825006542,1305.487771,1721.326897,-415.839126,61.672572,23.345143,-174.322266,23827.466311,23167.392718,22507.319124,23167.392718
2023-02-07,23264.291016,23310.974609,22756.257812,22757.267578,27187964471,1229.176130,1622.896744,-393.720614,65.755517,31.511035,627.822266,23803.674808,23212.237165,22620.799522,23212.237165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-13,90298.710938,90647.570312,89800.992188,90257.796875,64237748110,-1249.905492,-1861.335956,611.430465,44.731956,-10.536088,-553.046875,94299.542288,90802.472098,87305.401909,90802.472098
2025-12-14,88175.179688,90498.109375,87634.937500,90296.437500,50465972205,-1380.345062,-1765.137777,384.792715,40.374697,-19.250606,-2219.132812,94392.774509,90643.962612,86895.150714,90643.962612
2025-12-15,86419.781250,89983.921875,85304.078125,88171.078125,45559514323,-1606.842662,-1733.478754,126.636092,37.152956,-25.694087,98.210938,94367.639440,90650.977679,86934.315917,90650.977679


In [36]:
# --- Create subplot layout ---
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[0.5, 0.25, 0.25],
    subplot_titles=[f"{ticker} Price & BB/SMA", "MACD", "RSI"]
)

# --- Row 1: Candlestick ---
fig.add_trace(
    go.Candlestick(
        x=plot_data.index,
        open=plot_data["Open"],
        high=plot_data["High"],
        low=plot_data["Low"],
        close=plot_data["Close"],
        name="Price"
    ),
    row=1, col=1
)

# Bollinger Bands
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["BBU"], name="BB Upper"), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["BBM"], name="BB Middle"), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["BBL"], name="BB Lower"), row=1, col=1)

# SMA
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["SMA"], name="SMA"), row=1, col=1)

# --- Row 2: MACD ---
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["MACD"], name="MACD"), row=2, col=1)
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["Signal"], name="Signal"), row=2, col=1)

# --- Row 3: RSI ---
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["RSI"], name="RSI"), row=3, col=1)

# --- Layout ---
fig.update_layout(
    title=f"{ticker} Technical Analysis",
    height=900,
    template="plotly_dark",
    xaxis=dict(rangeslider=dict(visible=False))
)


# RSI reference lines
fig.update_yaxes(range=[0, 100], row=3, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)
fig.show()



In [43]:
plot_data.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'MACD', 'Signal', 'Hist',
       'RSI', 'CMO', 'MOM', 'BBU', 'BBM', 'BBL', 'SMA', 'label'],
      dtype='object')

In [46]:
df = plot_data.copy()
df["label"] = np.where(df["Close"] < df["Close"].shift(1), "down","up")
df.loc[df.index[0],"Close"] = np.nan
df.label

Date
2023-02-03      up
2023-02-04    down
2023-02-05    down
2023-02-06    down
2023-02-07      up
              ... 
2025-12-13      up
2025-12-14    down
2025-12-15    down
2025-12-16      up
2025-12-17    down
Name: label, Length: 1049, dtype: object

In [69]:
df =df[["MACD", "RSI", "CMO","MOM", "BBM", "SMA", "label"]].copy()
data_train, data_test  = df.loc[:"2024-12-31", :], df.loc["2025-01-01":, :]
data_train_x, data_train_y = data_train.loc[:, :"SMA"], data_train.loc[:, "label"]
data_train_y = np.where(data_train_y=="up", 1, 0)
data_train_y


array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,

In [ ]:
def data_find(data_x, data_y):
    data_x_u = []
    data_y_u = []
    window = 6
    for i in range(window, len(data_x)):
        data_x_u.append(data_x[i - window: window, :])
        data_y_u.append(data_y[window])
    return data_x_u, data_y_u

X_train, y_train = data_find(data_train_x.values, data_train_y.values)


692